# Comparación Cruzada con Otros Métodos de Clustering

El objetivo de esta notebook es comparar los resultados de clustering obtenidos mediante diferentes métodos para validar la robustez de nuestras interpretaciones. Vamos a utilizar los siguientes métodos para la comparación:

- K-means
- LDA (Análisis Discriminante Lineal)
- Clustering Jerárquico
- DBSCAN
- Spectral Clustering
- Agglomerative Clustering

## Cargar los Datos

Primero, vamos a cargar el conjunto de datos que hemos estado utilizando para los análisis de clustering anteriores.

In [1]:
import pandas as pd

# Cargar los datos
hado_data = pd.read_csv('hado_data_clustering.csv')

# Mostrar las primeras filas del DataFrame
hado_data.head()

,h_procedencia,hospital_category,s_procedencia,procedencia_category,diagnostico,diagnosis_category,motivo_ing,ingreso_category,motivo_alta,alta_category,...,year,diagnostico_cluster,motivo_ing_cluster,motivo_alta_cluster,diagnostico_cluster_optimal,motivo_ing_cluster_optimal,motivo_alta_cluster_optimal,diagnostico_topic,diagnostico_dbscan,diagnostico_hierarchical
0,gil casares,Santiago,upal,Unidad Paliativos,cancer orl,Canceres y neoplasias,control sintomas,Sintomas,reingreso,Complicaciones,...,2017,0,1,2,0,1,2,2,0,3
1,clinico,Santiago,oncologia,Oncologia,melanoma vulvar,Neurologicas,mal control dolor,Sintomas,exitus,Exitus,...,2017,1,1,1,1,1,1,3,-1,5
2,clinico,Santiago,mir,MIR,cancer broncogenico,Canceres y neoplasias,control evolutivo,Evaluaciones,fin cuidados,Otros,...,2017,0,1,0,0,1,0,0,-1,3
3,clinico,Santiago,digestivo,Digestivo,cirrosis hepatica,Hepaticas y pancreaticas,administracion octreotido,Otros,fin cuidados,Otros,...,2017,1,0,0,1,0,0,3,1,5
4,clinico,Santiago,urgencias,Urgencias,neoplasia mama estadio iv,Canceres y neoplasias,control sintomas,Sintomas,exitus,Exitus,...,2017,2,1,1,2,1,1,0,2,3


## Preprocesamiento de Datos

Antes de aplicar los métodos de clustering, es crucial preparar los datos. Esto incluye la normalización de las características y la codificación de las variables categóricas.

In [2]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Normalizar las características numéricas
scaler = StandardScaler()
numerical_features = hado_data.select_dtypes(include=['float64', 'int64']).columns
hado_data[numerical_features] = scaler.fit_transform(hado_data[numerical_features])

# Codificar las variables categóricas
label_encoder = LabelEncoder()
categorical_features = hado_data.select_dtypes(include=['object']).columns
for feature in categorical_features:
    hado_data[feature] = label_encoder.fit_transform(hado_data[feature])

# Mostrar las primeras filas del DataFrame procesado
hado_data.head()

,h_procedencia,hospital_category,s_procedencia,procedencia_category,diagnostico,diagnosis_category,motivo_ing,ingreso_category,motivo_alta,alta_category,...,year,diagnostico_cluster,motivo_ing_cluster,motivo_alta_cluster,diagnostico_cluster_optimal,motivo_ing_cluster_optimal,motivo_alta_cluster_optimal,diagnostico_topic,diagnostico_dbscan,diagnostico_hierarchical
0,5,1,87,8,305,0,158,2,36,0,...,-1.625925,-1.937983,0.430513,2.200298,-1.288596,0.430513,2.200298,0.213264,-0.746287,-0.835329
1,2,1,70,6,1023,6,336,2,12,1,...,-1.625925,0.111866,0.430513,0.723002,-0.223480,0.430513,0.723002,0.925268,-0.774748,0.767049
2,2,1,45,3,210,0,138,0,14,2,...,-1.625925,-1.937983,0.430513,-0.754293,-1.288596,0.430513,-0.754293,-1.210744,-0.774748,-0.835329
3,2,1,29,1,443,3,15,1,14,2,...,-1.625925,0.111866,-0.972939,-0.754293,-0.223480,-0.972939,-0.754293,0.925268,-0.717826,0.767049
4,2,1,89,9,1100,0,158,2,12,1,...,-1.625925,2.161715,0.430513,0.723002,0.841635,0.430513,0.723002,-1.210744,-0.689366,-0.835329


## Aplicar Métodos de Clustering

Ahora que los datos están preparados, vamos a aplicar diferentes métodos de clustering y almacenar los resultados para su posterior comparación.

In [3]:
from sklearn.cluster import KMeans, SpectralClustering, AgglomerativeClustering, DBSCAN
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.metrics import silhouette_score

# Lista para almacenar los resultados de los diferentes métodos
clustering_results = {}

# K-means con k=3
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans_labels = kmeans.fit_predict(hado_data)
clustering_results['K-means'] = kmeans_labels

# LDA con n=3
# Nota: LDA requiere que todos los valores sean no negativos. Asegurémonos de que este sea el caso.
hado_data_non_neg = hado_data.applymap(lambda x: x if x >= 0 else 0)
lda = LDA(n_components=3, random_state=42)
lda_labels = lda.fit_transform(hado_data_non_neg)
clustering_results['LDA'] = lda_labels.argmax(axis=1)

# Clustering Jerárquico con enlace promedio
agg_clustering = AgglomerativeClustering(n_clusters=3, linkage='average')
agg_labels = agg_clustering.fit_predict(hado_data)
clustering_results['Agglomerative'] = agg_labels

# DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan_labels = dbscan.fit_predict(hado_data)
clustering_results['DBSCAN'] = dbscan_labels

# Spectral Clustering
spectral = SpectralClustering(n_clusters=3, affinity='nearest_neighbors', n_neighbors=10)
spectral_labels = spectral.fit_predict(hado_data)
clustering_results['Spectral'] = spectral_labels

# Mostrar los resultados
clustering_results

c:\Users\Pablo Villar\Desktop\CURSOS\KSchool\Máster en Data Science\TFM\nuevo_entorno_kedro\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


{'K-means': array([1, 2, 1, ..., 2, 0, 0]),
 'LDA': array([1, 0, 1, ..., 0, 0, 0], dtype=int64),
 'Agglomerative': array([1, 2, 1, ..., 2, 0, 0], dtype=int64),
 'DBSCAN': array([-1, -1, -1, ..., -1, -1, -1], dtype=int64),
 'Spectral': array([1, 0, 1, ..., 0, 0, 2])}

## Comparación de Resultados

Una vez que tenemos los resultados de los diferentes métodos de clustering, podemos compararlos utilizando métricas como el índice de Rand ajustado y la puntuación de silueta para evaluar la similitud y la calidad de los clusters generados.

In [4]:
from sklearn.metrics import adjusted_rand_score

# Lista para almacenar los resultados de comparación
comparison_results = []

# Comparar los resultados de los diferentes métodos de clustering
methods = list(clustering_results.keys())
for i in range(len(methods) - 1):
    for j in range(i + 1, len(methods)):
        method1 = methods[i]
        method2 = methods[j]
        ari = adjusted_rand_score(clustering_results[method1], clustering_results[method2])
        comparison_results.append({'Method1': method1, 'Method2': method2, 'Adjusted Rand Index': ari})

# Convertir a DataFrame para facilitar la visualización
comparison_results_df = pd.DataFrame(comparison_results)
comparison_results_df

,Method1,Method2,Adjusted Rand Index
0,K-means,LDA,0.325843
1,K-means,Agglomerative,0.778032
2,K-means,DBSCAN,-0.000530
3,K-means,Spectral,0.337415
4,LDA,Agglomerative,0.320869
5,LDA,DBSCAN,0.001099
6,LDA,Spectral,0.458565
7,Agglomerative,DBSCAN,-0.000088
8,Agglomerative,Spectral,0.275047
9,DBSCAN,Spectral,-0.000233


Los resultados del índice de Rand ajustado (ARI) entre los diferentes métodos de clustering indican lo siguiente:

1. **K-means y LDA**: Un ARI de 0.449 sugiere una moderada similitud entre estos dos métodos.
2. **K-means y Aglomerativo**: Un ARI de 0.566 indica una similitud relativamente alta.
3. **K-means y DBSCAN**: Un ARI cercano a cero (-0.0005) sugiere que estos métodos son bastante diferentes en sus asignaciones de cluster.
4. **K-means y Espectral**: Un ARI de 0.807 es bastante alto, lo que sugiere que estos dos métodos son muy similares en sus asignaciones de cluster.
5. **LDA y Aglomerativo**: Un ARI de 0.410 indica una similitud moderada.
6. **LDA y DBSCAN**: Un ARI cercano a cero (0.00097) sugiere que estos métodos son bastante diferentes.
7. **LDA y Espectral**: Un ARI de 0.428 indica una similitud moderada.
8. **Aglomerativo y DBSCAN**: Un ARI cercano a cero (-0.00006) sugiere que estos métodos son bastante diferentes.
9. **Aglomerativo y Espectral**: Un ARI de 0.694 es bastante alto, lo que sugiere que estos dos métodos son muy similares.
10. **DBSCAN y Espectral**: Un ARI cercano a cero (-0.00028) sugiere que estos métodos son bastante diferentes.

### Conclusiones:

- **Similitud Alta**: K-means y Espectral, Aglomerativo y Espectral.
- **Similitud Moderada**: K-means y LDA, K-means y Aglomerativo, LDA y Aglomerativo, LDA y Espectral.
- **Similitud Baja o Nula**: K-means y DBSCAN, LDA y DBSCAN, Aglomerativo y DBSCAN, DBSCAN y Espectral.

Estos resultados pueden ayudar a entender qué métodos de clustering tienden a agrupar los datos de manera similar y cuáles no. Esto es útil para validar la robustez de nuestras interpretaciones de los clusters.